# Max Speed Analysis

Overall goal: get parameters on the max speed for some gloves and setup.

"Calculate for each stroke from 1 to 25 for the Right side, Left side, and the average of Right and Left."

For each parameter, calculate this only on the "contact time"
1. Total Force (N) = SQRT( tangential force² + radial force² + axle force² )
2. Tangential Force (N)
3. Radial force (N)
4. Axel force (N)
5. Torque (moment)
6. Power (W)

For the parameters 2-6, split into the positive and negative numbers to get these speeds

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the data

csv_files = ["ALP", "BS", "DR", "EC", "HD", "JF", "JR", "SS2"]
file_suffix = "25HYB.csv"

base_directory = "/Users/katieli/Research/wheelchair/data/raw/Max/"
hyp = "HYB"
pla = "PLA"

example_file = f"{base_directory}{hyp}/{csv_files[0]}{file_suffix}"

In [3]:
example_file

'/Users/katieli/Research/wheelchair/data/raw/Max/HYB/ALP25HYB.csv'

In [4]:
raw_df = pd.read_csv(example_file)

## Parameter 1 - total force

In [5]:
# Remove empty columns, and unused data
df = raw_df.drop(columns=[c for c in raw_df.columns if "Unnamed" in c])

# the angle measurement is NAN when the gloves are not touching the wheel
df = df[df['theta_cop_R[deg]'].notna()]

In [6]:
# calculate the total force parameter for both sides
df['total_force_R[N]'] = np.sqrt(
    df["tangential_force_R[N]"]**2 +
    df["radial_force_R[N]"]**2 +
    df["axle_force_R[N]"]**2
)

df['total_force_L[N]'] =  np.sqrt(
    df["tangential_force_L[N]"]**2 +
    df["radial_force_L[N]"]**2 +
    df["axle_force_L[N]"]**2
)

# Set missing values to 0
# i.e. if they lift their hand up earlier, the force is not recorded, and is set to 0
df['total_force_R[N]'] = df['total_force_R[N]'].fillna(0)
df['total_force_L[N]'] = df['total_force_L[N]'].fillna(0)

df['total_force_average[N]'] = (df['total_force_R[N]'] + df['total_force_L[N]'])/2.0

result = df.groupby("cycle[count]")[
    ['total_force_R[N]', 
     'total_force_L[N]',
     'total_force_average[N]']].mean().reset_index()

In [7]:
df[df["cycle[count]"]==3][['total_force_R[N]', 'total_force_L[N]']]

,total_force_R[N],total_force_L[N]
679,58.569329,28.959065
680,32.539032,39.721509
681,38.117318,31.984369
682,50.669041,59.183512
683,58.304694,45.084657
...,...,...
776,22.734133,41.052564
777,23.772231,36.779731
778,19.789454,32.233453
779,19.948432,28.131635


In [8]:
# sanity check that these are close numbers
#result['total_force_average[N]']- (result['total_force_R[N]'] + result['total_force_L[N]'])/2 
any(result['total_force_average[N]']- (result['total_force_R[N]'] + result['total_force_L[N]'])/2 > 0.05)

False

In [9]:
# did this significantly change anything? they lift their hand up earlier, 2% of the time
(df['total_force_L[N]']==0).sum()
df.describe()

45/1701*100

2.6455026455026456

## Parameter 2: tangential force
- split into positive, negative numbers
- output will be the columns:
  - `tangential_force_L_pos[N]`
  - `tangential_force_L_neg[N]`
  - `tangential_force_R_pos[N]`
  - `tangential_force_R_neg[N]`

In [17]:
# custom aggregation functions

# `x` is the series
def average_positive(x):
    positives = x[x >= 0]
    if len(x) == 0:
        return np.nan
    return positives.mean()


def average_negative(x):
    negatives = x[x < 0]
    if len(x) == 0:
        return np.nan
    return negatives.mean()

In [23]:
# add these columns

df.groupby('cycle[count]').agg(
    **{
        # right side
        "tangential_force_R_pos[N]": ("tangential_force_R[N]", average_positive),
        "tangential_force_R_neg[N]": ("tangential_force_R[N]", average_negative),
        # left side
        "tangential_force_L_pos[N]": ("tangential_force_L[N]", average_positive),
        "tangential_force_L_neg[N]": ("tangential_force_L[N]", average_negative),
    }
).reset_index()

,cycle[count],tangential_force_R_pos[N],tangential_force_R_neg[N],tangential_force_L_pos[N],tangential_force_L_neg[N]
0,1,121.493129,-9.282187,146.573813,-1.666547
1,2,136.674338,-9.093999,153.343885,-6.267965
2,3,161.424376,-12.898395,160.416214,-11.487418
3,4,143.657617,-0.225524,163.299298,-1.239371
4,5,154.858555,-5.744266,150.479478,-2.778149
5,6,146.172073,-8.064750,157.328730,-15.233110
6,7,132.089657,-6.316005,148.601245,-19.926093
7,8,130.901937,-12.351307,146.490366,NaN
8,9,136.818896,-8.092090,143.248690,NaN
9,10,144.534172,-8.304765,139.822823,-6.095366
